In [9]:
import numpy as np
from MCTS import MCTS
from Game import Game 

In [10]:
#test_probs = np.ones( (15,15) )
#test_probs /= 225.0


test_probs = np.ones( (15,15) )
for x in range(15):
    for y in range(15):
        test_probs[x,y] = 1.0 / (abs(7.0-x) + abs(7.0-y) + 1.0)**3
test_probs /= sum(sum(test_probs))


In [11]:
def evaluator(g):
    return test_probs, 0.5

g = Game()
nr=0
while (g.winner==0):
    if nr%2==0:
        x = np.random.randint(15)
        y = np.random.randint(15)
        while g.grid[x,y]!=0:
            x = np.random.randint(15)
            y = np.random.randint(15)
        g.move(x,y)
        g.print()
    else:
        mcts = MCTS(g, evaluator)
        x,y,_ = mcts.select_move()
        g.move(x,y)
        g.print() 
    nr+=1
print(nr, g.winner)


-----------------
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|          X    |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|          X    |
|               |
|       O       |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|              X|
|               |
|               |
|               |
|               |
|          X    |
|               |
|       O       |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|         

In [12]:
def evaluator(g):
    return test_probs, 0.5

g = Game()
n = None
nr = 0
while (g.winner==0):
    mcts = MCTS(g, evaluator, n)
    x,y,n = mcts.select_move()
    g.move(x,y)
    g.print() 
    nr+=1
print(nr,g.winner)


-----------------
|               |
|               |
|               |
|               |
|               |
|               |
|       X       |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|               |
|               |
|       X       |
|       O       |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|               |
|   X           |
|       X       |
|       O       |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|         